In [1]:
import pandas as pd
import chardet
import numpy as np

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter

import geopy.distance

import warnings
warnings.filterwarnings('ignore')

### Importing property data

In [2]:
property_dummy = pd.read_csv('property locations.csv')

In [3]:
property_df = property_dummy[['Market', 'Owner/Operator, Franchise', 
                              'ADDRESS', 'CTY', 'ST', 'Zip','Area', 'Year']]

property_df.rename(columns={'Market':'market', 'Owner/Operator, Franchise':'owner', 
                              'ADDRESS':'address', 'CTY':'city', 'ST':'state',
                            'Zip':'zip','Area':'area', 'Year':'year'},inplace=True)
property_df.head()

,market,owner,address,city,state,zip,area,year
0,Albuquerque,Extra Space Storage,1522 Pacheco Street,Santa Fe,NM,87505,73934.0,2000.0
1,Albuquerque,Extra Space Storage,1909 Golf Course Road SE,Rio Rancho,NM,87124,72836.0,2000.0
2,Albuquerque,Extra Space Storage,3300 Calle Cuervo NW,Albuquerque,NM,87114,80889.0,1998.0
3,Albuquerque,Extra Space Storage,9831 Montgomery Blvd NE,Albuquerque,NM,87111,62697.0,1997.0
4,Albuquerque,Extra Space Storage,9221 Eagle Ranch Road NW,Albuquerque,NM,87114,60821.0,1998.0


In [4]:
#### Data Cleaning

In [5]:
#### Removing extraspace from categorial columns and converting to lowercase

In [6]:
property_df['state'] = property_df['state'].str.strip()
property_df['market'] = property_df['market'].str.lower().str.strip()
property_df['city'] = property_df['city'].str.lower().str.strip()
property_df['owner'] = property_df['owner'].str.lower().str.strip()
property_df['address'] = property_df['address'].str.lower().str.strip()

In [7]:
#### Checking if there is any null value present in dataset

In [8]:
property_df.isnull().sum()

market      0
owner       0
address     0
city        0
state       0
zip         0
area        4
year       76
dtype: int64

In [9]:
#### Drop the records with area null

In [10]:
property_df[property_df['area'].isnull() == True]

,market,owner,address,city,state,zip,area,year
1506,new jersey - northern,cubesmart self storage,110 pleasant avenue,upper saddle river,NJ,7458,NaN,NaN
1734,baltimore,ez storage phase ii,1450 taylor avenue,parkville,MD,21234,NaN,NaN
1783,baltimore,ez storage phase ii,1450 taylor avenue,parkville,MD,21234,NaN,NaN
4128,seattle,public storage phase ii,1800 124th avenue ne,bellevue,WA,98005,NaN,NaN


In [11]:
property_df = property_df[property_df['area'].isnull() == False]

In [12]:
#### Checking if duplicate records present

In [13]:
property_df.duplicated().sum()

48

In [14]:
#### Removing duplicate entries

In [15]:
property_df.drop_duplicates(inplace=True)

In [16]:
property_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4379 entries, 0 to 4430
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   market   4379 non-null   object 
 1   owner    4379 non-null   object 
 2   address  4379 non-null   object 
 3   city     4379 non-null   object 
 4   state    4379 non-null   object 
 5   zip      4379 non-null   int64  
 6   area     4379 non-null   float64
 7   year     4309 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 307.9+ KB


In [17]:
property_df.describe()

,zip,area,year
count,4379.000000,4379.000000,4309.000000
mean,53983.802923,83526.052067,1993.154328
std,30460.940245,37405.805507,13.071374
min,1020.000000,4145.000000,1900.000000
25%,29613.000000,59242.000000,1985.000000
50%,53225.000000,77231.000000,1994.000000
75%,84105.500000,97946.000000,2002.000000
max,98685.000000,398279.000000,2021.000000


In [18]:
### 2. Create new dataframe from property dataframe by summing area value

In [19]:
storage_df = property_df.groupby(['zip'])['area'].sum().to_frame()
storage_df = storage_df.reset_index()
storage_df.head()

,zip,area
0,1020,72605.0
1,1129,89250.0
2,1301,49500.0
3,1453,71840.0
4,1501,159485.0


In [20]:
storage_df.duplicated().sum()

0

In [21]:
storage_df.isnull().sum()

zip     0
area    0
dtype: int64

In [22]:
len(storage_df)

2551

In [23]:
### 3. Merging Property and Storage dataframe on zipcode

In [24]:
property_new = property_df[['market','state','city', 'zip', ]]
property_new.head()

,market,state,city,zip
0,albuquerque,NM,santa fe,87505
1,albuquerque,NM,rio rancho,87124
2,albuquerque,NM,albuquerque,87114
3,albuquerque,NM,albuquerque,87111
4,albuquerque,NM,albuquerque,87114


In [25]:
df1 = storage_df.merge(property_new, how = 'left', left_on='zip',right_on='zip')
df1.head()

,zip,area,market,state,city
0,1020,72605.0,worcester - springfield,MA,chicopee
1,1129,89250.0,worcester - springfield,MA,springfield
2,1301,49500.0,worcester - springfield,MA,greenfield
3,1453,71840.0,worcester - springfield,MA,leominster
4,1501,159485.0,worcester - springfield,MA,auburn


In [26]:
df1.drop_duplicates(inplace=True)

In [27]:
len(df1)

2663

In [28]:
#### merging population and density data

In [29]:
population_df = pd.read_csv('dataset/population.csv',index_col=0)
population_df.head()

,zip,lat,lng,city,state_id,population,density,county
0,601,18.18004,-66.75218,adjuntas,PR,17242,111.4,adjuntas
1,602,18.36073,-67.17517,aguada,PR,38442,523.5,aguada
2,603,18.45439,-67.12202,aguadilla,PR,48814,667.9,aguadilla
3,606,18.16724,-66.93828,maricao,PR,6437,60.4,maricao
4,610,18.29032,-67.12243,anasco,PR,27073,312.0,anasco


In [30]:
df2 = df1.merge(population_df, how='left', left_on='zip',right_on='zip')
df2.head()

,zip,area,market,state,city_x,lat,lng,city_y,state_id,population,density,county
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,chicopee,MA,30097.0,914.6,hampden
1,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,springfield,MA,6762.0,790.8,hampden
2,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,greenfield,MA,17713.0,268.1,franklin
3,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,leominster,MA,41579.0,554.3,worcester
4,1501,159485.0,worcester - springfield,MA,auburn,42.19720,-71.84534,auburn,MA,16549.0,401.4,worcester


In [31]:
len(df2)

2663

In [32]:
df2.isnull().sum()

zip           0
area          0
market        0
state         0
city_x        0
lat           2
lng           2
city_y        2
state_id      2
population    2
density       2
county        2
dtype: int64

In [33]:
df2[df2['population'].isnull() == True]

,zip,area,market,state,city_x,lat,lng,city_y,state_id,population,density,county
297,11249,81700.0,brooklyn,NY,brooklyn,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2034,84129,76600.0,salt lake city,UT,salt lake city,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
### substituting NAN values

In [35]:
df2.loc[df2['zip'] == 11249,'county' ]= 'kings'
df2.loc[df2['zip'] == 11249,'lat' ]= 40.6501038
df2.loc[df2['zip'] == 11249,'lng' ]= -73.9495823
df2.loc[df2['zip'] == 84129,'lat' ]= 40.7596198
df2.loc[df2['zip'] == 84129,'lng' ]= -111.8867975
df2.loc[df2['zip'] == 84129,'county' ]= 'salt lake'

In [36]:
df2.head()

,zip,area,market,state,city_x,lat,lng,city_y,state_id,population,density,county
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,chicopee,MA,30097.0,914.6,hampden
1,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,springfield,MA,6762.0,790.8,hampden
2,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,greenfield,MA,17713.0,268.1,franklin
3,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,leominster,MA,41579.0,554.3,worcester
4,1501,159485.0,worcester - springfield,MA,auburn,42.19720,-71.84534,auburn,MA,16549.0,401.4,worcester


In [37]:
#### substituting nan in population and density by county mean of respective

In [38]:
df2['population'] = df2['population'].fillna(df2.groupby('county')['population'].transform('mean'))
df2['density'] = df2['density'].fillna(df2.groupby("county")['density'].transform('mean'))

In [39]:
df2.drop(columns=['city_y','state_id'], inplace=True)

In [40]:
df2.head()

,zip,area,market,state,city_x,lat,lng,population,density,county
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden
1,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden
2,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin
3,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester
4,1501,159485.0,worcester - springfield,MA,auburn,42.19720,-71.84534,16549.0,401.4,worcester


In [41]:
### 4. Merging house income file with data

In [42]:
house_income_df = pd.read_csv('dataset/income.csv',index_col=0)
house_income_df.head()

,state,county,city,zipcode,mean_income
0,AL,Mobile County,Chickasaw,36611,38773
1,AL,Barbour County,Louisville,36048,37725
2,AL,Shelby County,Columbiana,35051,54606
3,AL,Mobile County,Satsuma,36572,63919
4,AL,Mobile County,Dauphin Island,36528,77948


In [43]:
df3 = df2.merge(house_income_df, how='left', left_on=['zip'], right_on=['zipcode'])
df3.head()

,zip,area,market,state_x,city_x,lat,lng,population,density,county_x,state_y,county_y,city,zipcode,mean_income
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,MA,Barnstable County,Chicopee,1020.0,58782.0
1,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,MA,Barnstable County,Chicopee,1020.0,69193.0
2,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden,NaN,NaN,NaN,NaN,NaN
3,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin,MA,Barnstable County,Greenfield,1301.0,61561.0
4,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester,MA,Barnstable County,Leominster,1453.0,58855.0


In [44]:
len(df3)

8475

In [45]:
df3.drop_duplicates(inplace=True)

In [46]:
df3.columns

Index(['zip', 'area', 'market', 'state_x', 'city_x', 'lat', 'lng',
       'population', 'density', 'county_x', 'state_y', 'county_y', 'city',
       'zipcode', 'mean_income'],
      dtype='object')

In [47]:
df3.drop(columns=['state_y','city','zipcode','county_y'],inplace=True)
df3.rename(columns={'state_x':'state','city_x':'city','county_x':'county'},inplace=True)
df3.head()

,zip,area,market,state,city,lat,lng,population,density,county,mean_income
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,58782.0
1,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,69193.0
2,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden,NaN
3,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin,61561.0
4,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester,58855.0


In [48]:
df3.drop_duplicates(inplace=True)

In [49]:
df3.isnull().sum()

zip              0
area             0
market           0
state            0
city             0
lat              0
lng              0
population       0
density          0
county           0
mean_income    218
dtype: int64

In [50]:
### substituting nan mean_income with mean of county

In [51]:
df3['mean_income'] = df3['mean_income'].fillna(df3.groupby("county")['mean_income'].transform('mean'))

In [52]:
df3['mean_income'] = df3.groupby("zip")['mean_income'].transform('mean')

In [68]:
df3.shape

(2663, 11)

In [55]:
df3.drop_duplicates(inplace=True)

In [57]:
df3.head()

,zip,area,market,state,city,lat,lng,population,density,county,mean_income
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,63987.5
2,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden,63987.5
3,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin,61561.0
4,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester,55125.0
7,1501,159485.0,worcester - springfield,MA,auburn,42.19720,-71.84534,16549.0,401.4,worcester,77291.0


In [58]:
### 5. Merging house rent file with data

In [59]:
rent_df = pd.read_csv('dataset/rent.csv',index_col=0)
rent_df.head()

,state,county,city,zipcode,mean_rent
0,AL,chambers county,wadley,36276,972
1,AL,winston county,addison,35540,519
2,AL,marshall county,albertville,35950,625
3,AL,pickens county,aliceville,35442,546
4,AL,etowah county,walnut grove,35990,350


In [60]:
df4 = df3.merge(rent_df, how='left', left_on=['zip'], right_on=['zipcode'])
df4.head()

,zip,area,market,state_x,city_x,lat,lng,population,density,county_x,mean_income,state_y,county_y,city_y,zipcode,mean_rent
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,63987.5,MA,barnstable county,chicopee,1020.0,1087.0
1,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,63987.5,MA,barnstable county,chicopee,1020.0,890.0
2,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden,63987.5,NaN,NaN,NaN,NaN,NaN
3,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin,61561.0,MA,barnstable county,greenfield,1301.0,904.0
4,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester,55125.0,MA,barnstable county,leominster,1453.0,791.0


In [61]:
df4.columns

Index(['zip', 'area', 'market', 'state_x', 'city_x', 'lat', 'lng',
       'population', 'density', 'county_x', 'mean_income', 'state_y',
       'county_y', 'city_y', 'zipcode', 'mean_rent'],
      dtype='object')

In [62]:
df4.drop(columns=['state_y','city_y','zipcode','county_y'],inplace=True)
df4.rename(columns={'state_x':'state','city_x':'city','county_x':'county'},inplace=True)
df4.head()

,zip,area,market,state,city,lat,lng,population,density,county,mean_income,mean_rent
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,63987.5,1087.0
1,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,63987.5,890.0
2,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden,63987.5,NaN
3,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin,61561.0,904.0
4,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester,55125.0,791.0


In [63]:
df4.drop_duplicates(inplace=True)

In [64]:
df4['mean_rent'] = df4.groupby("zip")['mean_rent'].transform('mean')

In [65]:
df4.drop_duplicates(inplace=True)

In [66]:
df4.head()

,zip,area,market,state,city,lat,lng,population,density,county,mean_income,mean_rent
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,63987.5,988.500000
2,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden,63987.5,NaN
3,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin,61561.0,904.000000
4,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester,55125.0,917.666667
7,1501,159485.0,worcester - springfield,MA,auburn,42.19720,-71.84534,16549.0,401.4,worcester,77291.0,1063.000000


In [67]:
df4.isnull().sum()

zip              0
area             0
market           0
state            0
city             0
lat              0
lng              0
population       0
density          0
county           0
mean_income      8
mean_rent      209
dtype: int64

In [70]:
### substituting nan mean_income with mean of county

In [71]:
df4['mean_rent'] = df4['mean_rent'].fillna(df4.groupby("county")['mean_rent'].transform('mean'))

In [72]:
df4.shape

(2663, 12)

In [73]:
df4.head()

,zip,area,market,state,city,lat,lng,population,density,county,mean_income,mean_rent
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,63987.5,988.500000
2,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden,63987.5,988.500000
3,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin,61561.0,904.000000
4,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester,55125.0,917.666667
7,1501,159485.0,worcester - springfield,MA,auburn,42.19720,-71.84534,16549.0,401.4,worcester,77291.0,1063.000000


In [74]:
### Importing unemployment data

In [75]:
unemp_df = pd.read_csv('dataset/unemp.csv')
unemp_df.drop(columns=['Unnamed: 0'],inplace=True)

In [76]:
unemp_df.head()

,state,county,change_in_emprate
0,AL,autauga,-7.348113
1,AL,baldwin,-7.529913
2,AL,barbour,-7.045303
3,AL,bibb,-6.970139
4,AL,blount,-8.177910


In [77]:
df5 = df4.merge(unemp_df, how='left', left_on=['county','state'], right_on=['county','state'])
df5.head()

,zip,area,market,state,city,lat,lng,population,density,county,mean_income,mean_rent,change_in_emprate
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,63987.5,988.500000,0.846922
1,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden,63987.5,988.500000,0.846922
2,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin,61561.0,904.000000,1.197479
3,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester,55125.0,917.666667,0.582158
4,1501,159485.0,worcester - springfield,MA,auburn,42.19720,-71.84534,16549.0,401.4,worcester,77291.0,1063.000000,0.582158


In [78]:
df5.columns

Index(['zip', 'area', 'market', 'state', 'city', 'lat', 'lng', 'population',
       'density', 'county', 'mean_income', 'mean_rent', 'change_in_emprate'],
      dtype='object')

In [79]:
df5.isnull().sum()

zip                    0
area                   0
market                 0
state                  0
city                   0
lat                    0
lng                    0
population             0
density                0
county                 0
mean_income            8
mean_rent              7
change_in_emprate    441
dtype: int64

In [80]:
df5.shape

(2719, 13)

In [81]:
df5.duplicated().sum()

0

In [82]:
### substituting null value in change_in_emprate with mean of the state

In [83]:
df5['change_in_emprate'] = df5['change_in_emprate'].fillna(df5.groupby("state")['change_in_emprate'].transform('mean'))

In [84]:
df5[df5['mean_income'].isnull() == True]

,zip,area,market,state,city,lat,lng,population,density,county,mean_income,mean_rent,change_in_emprate
448,20619,82536.0,washington dc - suburban maryland,MD,california,38.29102,-76.52684,13533.0,258.7,st. mary's,NaN,NaN,-0.780992
449,20653,89658.0,washington dc - suburban maryland,MD,lexington park,38.23022,-76.43376,23546.0,289.3,st. mary's,NaN,NaN,-0.780992
671,27520,77660.0,raleigh - durham,NC,clayton,35.61427,-78.47188,40611.0,192.3,johnston,NaN,1073.0,-0.789758
1173,36542,59136.0,mobile,AL,gulf shores,30.27726,-87.73714,13932.0,93.4,baldwin,NaN,NaN,-7.529913
1315,46112,82464.0,indianapolis,IN,brownsburg,39.86506,-86.38258,37047.0,324.6,hendricks,NaN,NaN,-0.946561
1657,64153,139334.0,kansas city,MO,kansas city,39.28051,-94.73228,5234.0,56.5,platte,NaN,NaN,1.160880
2006,79762,93380.0,midland - odessa,TX,odessa,31.92649,-102.35443,43881.0,475.8,ector,NaN,NaN,-14.867547
2007,79764,70115.0,midland - odessa,TX,odessa,31.89055,-102.47102,21685.0,137.4,ector,NaN,NaN,-14.867547


In [85]:
### substituting Nan in change_in_emprate with mean

In [86]:
df5['change_in_emprate'].fillna((df5['change_in_emprate'].mean()), inplace=True)

In [87]:
df5['mean_rent'].fillna((df5['mean_rent'].mean()), inplace=True)

In [88]:
df5['mean_income'].fillna((df5['mean_income'].mean()), inplace=True)

In [89]:
df5

,zip,area,market,state,city,lat,lng,population,density,county,mean_income,mean_rent,change_in_emprate
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,63987.500000,988.500000,0.846922
1,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden,63987.500000,988.500000,0.846922
2,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin,61561.000000,904.000000,1.197479
3,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester,55125.000000,917.666667,0.582158
4,1501,159485.0,worcester - springfield,MA,auburn,42.19720,-71.84534,16549.0,401.4,worcester,77291.000000,1063.000000,0.582158
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2714,98516,100590.0,tacoma,WA,lacey,47.09657,-122.78360,24536.0,271.8,thurston,93946.666667,1315.000000,0.193800
2715,98662,115102.0,portland,WA,vancouver,45.68850,-122.57784,32430.0,949.0,clark,70668.000000,1272.500000,-1.151240
2716,98664,57000.0,portland,WA,vancouver,45.61981,-122.57752,22873.0,1597.8,clark,58808.000000,1052.000000,-1.151240
2717,98665,61250.0,portland,WA,vancouver,45.67977,-122.65933,26556.0,1152.8,clark,72082.250000,1433.500000,-1.151240


In [90]:
df5.isnull().sum()

zip                  0
area                 0
market               0
state                0
city                 0
lat                  0
lng                  0
population           0
density              0
county               0
mean_income          0
mean_rent            0
change_in_emprate    0
dtype: int64

In [91]:
df5.duplicated().sum()

0

In [92]:
df5.to_csv('dataset/complete_storage.csv')

In [93]:
df6 = df5

In [94]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter

In [95]:
df6.head()

,zip,area,market,state,city,lat,lng,population,density,county,mean_income,mean_rent,change_in_emprate
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,63987.5,988.500000,0.846922
1,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden,63987.5,988.500000,0.846922
2,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin,61561.0,904.000000,1.197479
3,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester,55125.0,917.666667,0.582158
4,1501,159485.0,worcester - springfield,MA,auburn,42.19720,-71.84534,16549.0,401.4,worcester,77291.0,1063.000000,0.582158


In [96]:
locator = Nominatim(user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

In [97]:
market_list = list(df6['market'].unique())

In [98]:
mark_lat = dict()
mark_long = dict()
for i in range(len(market_list)):
    
    #print(market_list[i])
    locate = geocode(market_list[i],timeout=10)
    #print(locate.latitude)
    if locate is None:
        locate = geocode(market_list[i].split(' ')[0],timeout=10)
    
    mark_lat[market_list[i]] = locate.latitude
    mark_long[market_list[i]] = locate.longitude

In [99]:
df6['market_latitude'] = df6['market'].map(mark_lat)
df6['market_longitude'] = df6['market'].map(mark_long)
df6.head()

,zip,area,market,state,city,lat,lng,population,density,county,mean_income,mean_rent,change_in_emprate,market_latitude,market_longitude
0,1020,72605.0,worcester - springfield,MA,chicopee,42.17637,-72.56537,30097.0,914.6,hampden,63987.5,988.500000,0.846922,52.198987,-2.22679
1,1129,89250.0,worcester - springfield,MA,springfield,42.12099,-72.48774,6762.0,790.8,hampden,63987.5,988.500000,0.846922,52.198987,-2.22679
2,1301,49500.0,worcester - springfield,MA,greenfield,42.62406,-72.60401,17713.0,268.1,franklin,61561.0,904.000000,1.197479,52.198987,-2.22679
3,1453,71840.0,worcester - springfield,MA,leominster,42.51863,-71.76138,41579.0,554.3,worcester,55125.0,917.666667,0.582158,52.198987,-2.22679
4,1501,159485.0,worcester - springfield,MA,auburn,42.19720,-71.84534,16549.0,401.4,worcester,77291.0,1063.000000,0.582158,52.198987,-2.22679


In [100]:
df6[df6['market_longitude'].isnull() == True]

,zip,area,market,state,city,lat,lng,population,density,county,mean_income,mean_rent,change_in_emprate,market_latitude,market_longitude


In [230]:
def map_togeocode_series(address):
    lat = []
    longi = []
    for i in range(len(address)):
        #print(address[i])
        city = [address[i].split('-')][0][0]
        city = city.strip()
        #add = add.strip()
        #print(city, add)
        locator = Nominatim(user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9')
        geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
        locate = geocode(city,timeout=10)
        '''if locate is None:
            locate = geocode(city)'''
        lat.append(locate.latitude)
        longi.append(locate.longitude)
    return(lat,longi)

In [101]:
df7 = df6

In [102]:
df6.to_csv('dataset/storage_market_coord.csv')

In [106]:
df6['city_coord'] = df6['lat'].astype(str) + ',' + df6['lng'].astype(str)
df6['market_coord'] = df6['market_latitude'].astype(str) + ',' + df6['market_longitude'].astype(str)

In [107]:
def get_distance(coord1,coord2):
    #print(coord1)
    dist = geopy.distance.geodesic(coord1, coord2).km
    #print(dist)
    return dist

In [109]:
def haversine_vectorize(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    newlon = lon2 - lon1
    newlat = lat2 - lat1

    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2

    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    km = 6367 * dist #6367 for distance in KM for miles use 3958
    return km

In [110]:
df6['distance'] = haversine_vectorize(df6['lng'],df6['lat'],df6['market_longitude'],df6['market_latitude'])

In [111]:
df6['distance'].describe()

count     2719.000000
mean      1977.645691
std       4409.661230
min          0.000000
25%         13.587222
50%         25.084077
75%         80.653756
max      15997.029683
Name: distance, dtype: float64

In [112]:
df6.isnull().sum()

zip                  0
area                 0
market               0
state                0
city                 0
lat                  0
lng                  0
population           0
density              0
county               0
mean_income          0
mean_rent            0
change_in_emprate    0
market_latitude      0
market_longitude     0
city_coord           0
market_coord         0
distance             0
dtype: int64

In [114]:
df6.duplicated().sum()

0

In [115]:
df6.to_csv('dataset/complete_storage.csv')